<a href="https://colab.research.google.com/github/xuanmse/python-for-engineer/blob/master/Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Thông tin   |               |
| ----------- | -----------   |
| MSSV        | 22mse23076    |
| Họ Và Tên   | KIEU VAN XUAN |

#**Yêu cầu**

Cho tập dữ liệu về doanh số bán hàng gồm các thông tin như sau (https://drive.google.com/file/d/1QWD8rceAWo6KJYHh6Tx0eBtAM9U1EQno/view?usp=sharing(sad)
Superstore Sales Dataset Analysis | Kaggle
```
- order_id
- order_date
- ship_date
- ship_mode
- customer_id
- customer_name
- segment
- country
- city
- state
- region
- product_id
- sub-category
- product_name
- sales
```
Hãy sử dụng Python và Jupyter Notebook để trả lời các câu hỏi sau:
- YC1: Tiền xử lý dữ liệu (xử lý các trường hợp thiếu dữ liệu, chuẩn hóa ngày tháng, đơn vị,...).
- YC2: Vẽ biểu đồ thể hiện biến động doanh số theo tháng. Cho biết doanh thu trung bình trong 3 tháng gần nhất.
- YC3: Vẽ biểu đồ doanh số bán hàng theo quý và cho biết quý nào có tổng doanh số cao nhất?
- YC4: Tính thời gian trung giao hàng trung bình (ship date - order date) theo từng tháng và vẽ biểu đồ mô tả sự biến thiên của con số này.
- YC5: Vẽ biểu đồ thể hiện sức mua theo từng thành phố. Cho biết top 3 thành phố có doanh số thấp nhất và 3 thành phố có doanh số cao nhất.
- YC6: Vẽ biểu đồ thể hiện tỉ trọng doanh số theo bang (state) và cho biết trung bình doanh số của 5 bang dẫn đầu về sức mua.
- YC7: Cho biết các bang dẫn đầu doanh số theo từng vùng (east, west, south, central,..).
- YC8: Cho biết top 3 sản phẩm có doanh thu cao nhất và top 3 sản phẩm có số lượng đơn vị bán ra cao nhất.
- YC9: Vẽ biểu đồ thể hiện tỉ trọng doanh số của từng category.
- YC10: Dự báo doanh số theo nhiều khía cạnh khác nhau trong tương lai: theo vùng, theo bang, theo category,..  và phát hành API để khai thác.

#**Cài đặt môi trường & tải tập dữ liệu**

##**Cài đặt thư viện**

In [ ]:
!pip install kaggle # cài đặt công cụ Kaggle
!pip install seaborn
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##**Cài đặt xác thực**

In [ ]:
# Tải thông tin tài khoảng để xác thực trên kaggle và lưu vào máy ảo colab
! yes | mkdir /root/.kaggle
! yes | wget https://raw.githubusercontent.com/xuanmse/python-for-engineer/master/kaggle.json -P /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
--2022-11-01 16:37:39--  https://raw.githubusercontent.com/xuanmse/python-for-engineer/master/kaggle.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68 [text/plain]
Saving to: ‘/root/.kaggle/kaggle.json.1’

kaggle.json.1       100%[===================>]      68  --.-KB/s    in 0s      

2022-11-01 16:37:39 (1.84 MB/s) - ‘/root/.kaggle/kaggle.json.1’ saved [68/68]



##**Giải nén**

In [ ]:
!kaggle datasets download -d rohitsahoo/sales-forecasting -p /kaggle/input/
# !yes | kaggle competitions download -c m5-forecasting-accuracy -p /kaggle/input/
!yes | unzip /kaggle/input/sales-forecasting.zip -d /kaggle/input/sales-forecasting
!yes | rm /kaggle/input/sales-forecasting.zip

  0% 0.00/480k [00:00<?, ?B/s]
100% 480k/480k [00:00<00:00, 123MB/s]
Archive:  /kaggle/input/sales-forecasting.zip
replace /kaggle/input/sales-forecasting/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: /kaggle/input/sales-forecasting/train.csv  


##**Nhập thư viện và kiểm tra dữ liệu**

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split


##**Tải tập dữ liệu lên ô nhớ**

In [ ]:
# Reading dataset's ...
train = pd.read_csv('/kaggle/input/sales-forecasting/train.csv')

##**Kiểm tra dữ liệu đã tải lên**

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

#**YC1: Tiền xử lý dữ liệu**

##**Kiểm tra dữ liệu rổng**

In [ ]:
train.isnull().sum().sort_values(ascending = False)

Postal Code      11
Row ID            0
Order ID          0
Product Name      0
Sub-Category      0
Category          0
Product ID        0
Region            0
State             0
City              0
Country           0
Segment           0
Customer Name     0
Customer ID       0
Ship Mode         0
Ship Date         0
Order Date        0
Sales             0
dtype: int64

Thiếu dữ liệu về postal code, sử dụng thông tin thành phố để điền cập nhật lại

##**Cập nhật dữ liệu post code**

In [ ]:
train[train['Postal Code'].isnull()==True]['City'].unique()

array(['Burlington'], dtype=object)

In [ ]:
train[train['City']=='Burlington']['Postal Code'].value_counts()
train['Postal Code'] = train['Postal Code'].fillna(27217)
train['Postal Code'] = train['Postal Code'].apply(int)
train.isnull().sum().sort_values(ascending = False)

Row ID           0
Order ID         0
Product Name     0
Sub-Category     0
Category         0
Product ID       0
Region           0
Postal Code      0
State            0
City             0
Country          0
Segment          0
Customer Name    0
Customer ID      0
Ship Mode        0
Ship Date        0
Order Date       0
Sales            0
dtype: int64

In [ ]:
train

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


##**Cập nhật tên cột**


In [ ]:
train.rename(columns={'Row ID':'Row_ID',
                   'Order ID': 'Order_ID',
                   'Order Date':'Order_Date',
                   'Ship Date':'Ship_Date',
                   'Customer ID':'Custmoer_ID',
                   'Customer Name':'Customer_Name',
                   'Postal Code':'Postal_Code',
                   'Product ID':'Product_ID',
                   'Sub-Category':'Sub_Category',
                   'Product Name':'Product_Name'},inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row_ID         9800 non-null   int64  
 1   Order_ID       9800 non-null   object 
 2   Order_Date     9800 non-null   object 
 3   Ship_Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Custmoer_ID    9800 non-null   object 
 6   Customer_Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal_Code    9800 non-null   int64  
 12  Region         9800 non-null   object 
 13  Product_ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub_Category   9800 non-null   object 
 16  Product_Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

##**Cập nhận dữ liệu ngày tháng**

In [ ]:
train['Order_Date'] = pd.to_datetime(train['Order_Date'], dayfirst=True)
train['Ship_Date'] = pd.to_datetime(train['Ship_Date'],dayfirst=True)
train

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship Mode,Custmoer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,2017-05-21,2017-05-28,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,2016-01-12,2016-01-17,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,2016-01-12,2016-01-17,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,2016-01-12,2016-01-17,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


#**YC2: Vẽ biểu đồ thể hiện biến động doanh số theo tháng. Cho biết doanh thu trung bình trong 3 tháng gần nhất.**

Tính toán dữ liệu

In [ ]:
g = train
per = train.Order_Date.dt.to_period("M")
g = g.groupby(per)["Sales"].sum()
g

Order_Date
2015-01     14205.7070
2015-02      4519.8920
2015-03     55205.7970
2015-04     27906.8550
2015-05     23644.3030
2015-06     34322.9356
2015-07     33781.5430
2015-08     27117.5365
2015-09     81623.5268
2015-10     31453.3930
2015-11     77907.6607
2015-12     68167.0585
2016-01     18066.9576
2016-02     11951.4110
2016-03     32339.3184
2016-04     34154.4685
2016-05     29959.5305
2016-06     23599.3740
2016-07     28608.2590
2016-08     36818.3422
2016-09     63133.6060
2016-10     31011.7375
2016-11     75249.3995
2016-12     74543.6012
2017-01     18542.4910
2017-02     22978.8150
2017-03     51165.0590
2017-04     38679.7670
2017-05     56656.9080
2017-06     39724.4860
2017-07     38320.7830
2017-08     30542.2003
2017-09     69193.3909
2017-10     59583.0330
2017-11     79066.4958
2017-12     95739.1210
2018-01     43476.4740
2018-02     19920.9974
2018-03     58863.4128
2018-04     35541.9101
2018-05     43825.9822
2018-06     48190.7277
2018-07     44825.1040


In [ ]:
fig = px.bar(g, x=g.index.strftime('%b-%Y'),y = "Sales")
fig.show()

In [ ]:
fig = px.line(g, x=g.index.strftime('%b-%Y'),y = "Sales")
fig.show()

Doanh thu trung bình trong 3 tháng gần nhất.

In [ ]:
g = train.groupby(per)["Sales"].mean()
g[-3:]

Order_Date
2018-10    263.429018
2018-11    258.636305
2018-12    180.894093
Freq: M, Name: Sales, dtype: float64

#**YC3: Vẽ biểu đồ doanh số bán hàng theo quý và cho biết quý nào có tổng doanh số cao nhất?**

In [ ]:
per = train.Order_Date.dt.to_period("Q")
g = train
g = g.groupby(per)["Sales"].sum()
g

Order_Date
2015Q1     73931.3960
2015Q2     85874.0936
2015Q3    142522.6063
2015Q4    177528.1122
2016Q1     62357.6870
2016Q2     87713.3730
2016Q3    128560.2072
2016Q4    180804.7382
2017Q1     92686.3650
2017Q2    135061.1610
2017Q3    138056.3742
2017Q4    234388.6498
2018Q1    122260.8842
2018Q2    127558.6200
2018Q3    193815.8400
2018Q4    278416.6750
Freq: Q-DEC, Name: Sales, dtype: float64

In [ ]:
fig = px.bar(g, x=g.index.strftime('%q-%Y'),y = "Sales")
fig.show()

Quý có doanh thu cao nhất:

In [ ]:
# Quý có doanh số cao nhất
F  = g.sort_values(ascending=False)
print(F.index[0], F.values[0])
# fig = px.bar(g, x=g.index.to_timestamp(),y = "Row_ID")
# fig.show()

2018Q4 278416.675


#**YC4: Tính thời gian trung giao hàng trung bình (ship date - order date) theo từng tháng và vẽ biểu đồ mô tả sự biến thiên của con số này.**


In [ ]:
X = train 
X["Deliver_time"] = X["Ship_Date"] - X["Order_Date"] 

per = X.Order_Date.dt.to_period("M")
g = X.groupby(per)["Deliver_time"].mean()/ np.timedelta64(1, 'D') # thời gian theo giờ
g


Order_Date
2015-01    4.194805
2015-02    4.173913
2015-03    3.961039
2015-04    4.015385
2015-05    3.801653
2015-06    4.213740
2015-07    3.845070
2015-08    4.089041
2015-09    4.056180
2015-10    3.358491
2015-11    4.248366
2015-12    3.959854
2016-01    4.736842
2016-02    4.218750
2016-03    3.765625
2016-04    4.333333
2016-05    4.020979
2016-06    4.162791
2016-07    3.786765
2016-08    3.955696
2016-09    4.340351
2016-10    3.609756
2016-11    3.978193
2016-12    3.938907
2017-01    4.101124
2017-02    3.469880
2017-03    3.503106
2017-04    4.452381
2017-05    3.877828
2017-06    3.413265
2017-07    3.836735
2017-08    4.331429
2017-09    3.977077
2017-10    4.203125
2017-11    3.948087
2017-12    4.014793
2018-01    4.062937
2018-02    4.355769
2018-03    3.578059
2018-04    3.860000
2018-05    3.725000
2018-06    4.080851
2018-07    4.130045
2018-08    4.042056
2018-09    3.693157
2018-10    3.996599
2018-11    3.800439
2018-12    4.028322
Freq: M, Name: Deliver_time, 

In [ ]:
fig = px.bar(g, x=g.index.strftime('%b-%Y'),y = "Deliver_time")
fig.show()

#**YC5: Vẽ biểu đồ thể hiện sức mua theo từng thành phố. Cho biết top 3 thành phố có doanh số thấp nhất và 3 thành phố có doanh số cao nhất.**

In [ ]:
g = train
g = g.groupby(["City"])["Sales"].sum()
g

City
Aberdeen         25.500
Abilene           1.392
Akron          2724.244
Albuquerque    2220.160
Alexandria     5519.570
                 ...   
Woonsocket      195.550
Yonkers        7657.666
York            817.978
Yucaipa          50.800
Yuma            840.865
Name: Sales, Length: 529, dtype: float64

In [ ]:
fig = px.bar(g)
fig.show()

In [ ]:
g  = g.sort_values(ascending=False)
print(g.head(3))

City
New York City    252462.547
Los Angeles      173420.181
Seattle          116106.322
Name: Sales, dtype: float64


In [ ]:
print(g.tail(3))

City
Jupiter    2.064
Elyria     1.824
Abilene    1.392
Name: Sales, dtype: float64


#**YC6: Vẽ biểu đồ thể hiện tỉ trọng doanh số theo bang (state) và cho biết trung bình doanh số của 5 bang dẫn đầu về sức mua.**

In [ ]:
g = train
fig = px.pie(g, values='Sales', names='State')
fig.show()

In [ ]:
g = g.groupby(["State"])["Sales"].sum().sort_values(ascending=False)

In [ ]:
g.head(5).mean()

234544.72853999998

#**YC7: Cho biết các bang dẫn đầu doanh số theo từng vùng (east, west, south, central,..).**

In [ ]:
g = train
g.groupby(["Region","State"])["Sales"].sum().sort_values().groupby(level=0).tail(1)

Region   State     
South    Florida        88436.5320
Central  Texas         168572.5322
East     New York      306361.1470
West     California    446306.4635
Name: Sales, dtype: float64

#**YC8: Cho biết top 3 sản phẩm có doanh thu cao nhất và top 3 sản phẩm có số lượng đơn vị bán ra cao nhất.**

Top3 sản phẩm có doanh thu cao nhất

In [ ]:
g = train
g.groupby("Product_ID")["Sales"].sum().sort_values(ascending=False).head(3)

Product_ID
TEC-CO-10004722    61599.824
OFF-BI-10003527    27453.384
TEC-MA-10002412    22638.480
Name: Sales, dtype: float64

Top 3 sản phẩm có đơn vị bán ra cao nhất

In [ ]:
g = train
g.groupby("Product_ID")["Row_ID"].sum().sort_values(ascending=False).head(3)

Product_ID
OFF-PA-10001970    86699
OFF-BI-10001524    83138
OFF-BI-10002026    81080
Name: Row_ID, dtype: int64

#**YC9: Vẽ biểu đồ thể hiện tỉ trọng doanh số của từng category.**

In [ ]:
g = train
g

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship Mode,Custmoer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Deliver_time
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,3 days
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3 days
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,4 days
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,7 days
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,7 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,2017-05-21,2017-05-28,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980,7 days
9796,9797,CA-2016-128608,2016-01-12,2016-01-17,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680,5 days
9797,9798,CA-2016-128608,2016-01-12,2016-01-17,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880,5 days
9798,9799,CA-2016-128608,2016-01-12,2016-01-17,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760,5 days


In [ ]:
fig = px.pie(g, values='Sales', names='Category')
fig.show()

#**YC10: Dự báo doanh số theo nhiều khía cạnh khác nhau trong tương lai: theo vùng, theo bang, theo category,.. và phát hành API để khai thác.**

Dự báo danh số bán hàng theo bang cho các ngày tiếp theo

##**Tạo data train**

In [ ]:
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

df = train.copy()
df = encode_dates(df, column='Order_Date')

df = df.groupby(["State","Order_Date_year","Order_Date_month","Order_Date_day"])["Sales"].sum()


State      Order_Date_year  Order_Date_month  Order_Date_day
Alabama    2015             4                 7                    8.960
                                              8                 1215.920
                            5                 22                 180.930
                            10                18                2216.800
                            11                28                  14.670
                                                                  ...   
Wisconsin  2018             11                27                 190.670
                            12                8                 1816.360
                                              17                  32.560
                                              18                 129.290
Wyoming    2017             11                7                 1603.136
Name: Sales, Length: 4147, dtype: float64

In [ ]:
X = pd.get_dummies(df.index.to_frame())
Y = df.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, shuffle=True, random_state=1)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4147 entries, ('Alabama', 2015, 4, 7) to ('Wyoming', 2017, 11, 7)
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Order_Date_year             4147 non-null   int64
 1   Order_Date_month            4147 non-null   int64
 2   Order_Date_day              4147 non-null   int64
 3   State_Alabama               4147 non-null   uint8
 4   State_Arizona               4147 non-null   uint8
 5   State_Arkansas              4147 non-null   uint8
 6   State_California            4147 non-null   uint8
 7   State_Colorado              4147 non-null   uint8
 8   State_Connecticut           4147 non-null   uint8
 9   State_Delaware              4147 non-null   uint8
 10  State_District of Columbia  4147 non-null   uint8
 11  State_Florida               4147 non-null   uint8
 12  State_Georgia               4147 non-null   uint8
 13  State_Idaho          

##**Định nghĩa model**

In [ ]:
import tensorflow as tf
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 52)]              0         
                                                                 
 dense_3 (Dense)             (None, 256)               13568     
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 79,617
Trainable params: 79,617
Non-trainable params: 0
_________________________________________________________________
None


##**Huấn luyện model**

In [ ]:
model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

Epoch 1/100
83/83 [==============================] - 1s 5ms/step - loss: 1234148.8750 - val_loss: 1075400.5000 - lr: 0.0010
Epoch 2/100
83/83 [==============================] - 0s 3ms/step - loss: 1239368.0000 - val_loss: 1051019.1250 - lr: 0.0010
Epoch 3/100
83/83 [==============================] - 0s 4ms/step - loss: 1231477.8750 - val_loss: 1061047.2500 - lr: 0.0010
Epoch 4/100
83/83 [==============================] - 0s 4ms/step - loss: 1230620.0000 - val_loss: 1090886.7500 - lr: 0.0010
Epoch 5/100
83/83 [==============================] - 0s 4ms/step - loss: 1227866.3750 - val_loss: 1064688.6250 - lr: 0.0010
Epoch 6/100
83/83 [==============================] - 0s 4ms/step - loss: 1231542.0000 - val_loss: 1048561.4375 - lr: 0.0010
Epoch 7/100
83/83 [==============================] - 0s 4ms/step - loss: 1229165.0000 - val_loss: 1052227.0000 - lr: 0.0010
Epoch 8/100
83/83 [==============================] - 0s 3ms/step - loss: 1231199.1250 - val_loss: 1051428.0000 - lr: 0.0010
Epoch 9/

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: {:.5f}".format(test_loss))

Test Loss: 936343.12500


In [ ]:
from sklearn.metrics import r2_score
y_pred = np.squeeze(model.predict(X_test))
test_r2 = r2_score(y_test, y_pred)

print("Test R^2 Score: {:.5f}".format(test_r2))

26/26 [==============================] - 0s 2ms/step
Test R^2 Score: -0.00018


##**Tạo API**

In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# flask_ngrok_example.py
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    state = request.args.get('state')
    y = request.args.get('y')
    m = request.args.get('m')
    d = request.args.get('d')
    return predict_data(state,y,m,d)

def predict_data(cus_test_state,cus_test_year,cus_test_month,cus_test_day ):

    key_test =pd.MultiIndex.from_tuples([(cus_test_state,cus_test_year,cus_test_month,cus_test_day)], names=["State","Order_Date_year", "Order_Date_month", "Order_Date_day"])
    X_test.columns
    ff = np.zeros(52,int)

    data_input = pd.DataFrame([ff],index=[key_test], columns = X_test.columns)
    data_input.iloc[0] = int(0)
    data_input["Order_Date_year"] = cus_test_year
    data_input["Order_Date_month"] = cus_test_month
    data_input["Order_Date_day"] = cus_test_day
    data_input["State_" + cus_test_state] = 1
    return model.predict(data_input)


if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1a63-35-225-86-128.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-89-de69abf3b2b7>", line 15, in hello
    return predict_data(state,y,m,d)
  File "<ipython-input-89-de69abf3b2b7>", 